In [ ]:
import numpy as np
import pandas as pd
from  skimage.io import imread, imshow
from knn import KNN
from svm import SVM
from reglog import Regresion
import pywt
import pywt.data
import os
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from PIL import Image
from pathlib import Path

def redimensionar(path):
   imagenes=[]
   entries = Path(path)
   for entry in entries.iterdir():
      imagenes.append(path + entry.name)

   # Tamaño deseado para todas las imágenes
   ancho_deseado = 300
   alto_deseado = 200

   # Carpeta de destino para las imágenes redimensionadas
   carpeta_destino = "imagenes_1"

   # Crea la carpeta de destino si no existe
   if not os.path.exists(carpeta_destino):
      os.makedirs(carpeta_destino)

   # Recorre la lista de imágenes y redimensiona y guarda cada una
   for ruta_imagen in imagenes:
      imagen = Image.open(ruta_imagen)
      
      # Redimensiona la imagen al tamaño deseado con interpolación ANTIALIAS
      imagen_redimensionada = imagen.resize((ancho_deseado, alto_deseado), Image.ADAPTIVE)
      
      # Obtiene el nombre de archivo sin la ruta
      nombre_archivo = os.path.basename(ruta_imagen)
      
      # Define la ruta de archivo de destino en la carpeta de destino
      ruta_destino = os.path.join(carpeta_destino, nombre_archivo)
      
      # Guarda la imagen redimensionada en la carpeta de destino
      imagen_redimensionada.save(ruta_destino)     

def Get_Feacture(picture, cortes):
  LL = picture
  for i in range(cortes):
     LL, (LH, HL, HH) = pywt.dwt2(LL, 'haar')
  return LL.flatten().tolist()

def cargar_dataset():

    carpeta_imagenes = './imagenes_1/'

    # Se ordena los archivos segun el nombre asignado
    archivos = os.listdir(carpeta_imagenes)

    clases = []
    vectores_caracteristicos = []


    for archivo in archivos:
        # Verifica si el archivo es una imagen (puedes agregar más extensiones si es necesario)
        if archivo.endswith(('.png')):
            # Construye la ruta completa del archivo
            ruta_completa = os.path.join(carpeta_imagenes, archivo)
            imagen = imread(ruta_completa)

            # Se añade la clase correspondiente al vector clases 
            if int(archivo[1]) < 1:
                clases.append(int(archivo[2]))
            else:
                clases.append(int(archivo[1:3]))

            # Se añade el vector caracteristico de cada imagen
            vectores_caracteristicos.append(Get_Feacture(imagen, 2))

    return vectores_caracteristicos,clases

def acuary(y_prueba,y_correct):
    correctos= np.sum(y_prueba == y_correct)
    return (correctos/len(y_correct))*100      

  
# Redimensionamos las imagenes para trabajar todas con el mismo tamaño
redimensionar("./images/")

# Definimos el dataset
x,y= cargar_dataset()

x=np.array(x)
y=np.array(y)

# Modelo Regresión Logistica

#### Validacion por division del dataset en 70% entrenamiento 15% validación, 15% prueba.

In [ ]:
# Definimos los tamaños de los conjuntos (70% entrenamiento, 15% validación, 15% prueba)
total_samples = len(x)
train = int(0.7 * total_samples)
validation = int(0.15 * total_samples)

# Dividimoss los datos en conjuntos de entrenamiento, validación y prueba
X_train = x[:train]
y_train = y[:train]

X_val = x[train:train + validation]
y_val = y[train:train + validation]

X_test = x[train + validation:]
y_test = y[train + validation:]

# Crear y entrenar el modelo
modelo = Regresion(1000,0.00000001)
modelo.train(X_train, y_train)

# Validacion
y_pred_val = modelo.predict(X_val)
precision_val=acuary(y_val,y_pred_val)

print(f"Precisión en el conjunto de validación: {precision_val:.2f}")

# Predicción
y_pred_test = modelo.predict(X_test)
precision_test=acuary(y_test,y_pred_test)

print(f"Precisión en el conjunto de testeo: {precision_test:.2f}")

#### Validacion por K-fold cross validation

In [ ]:
# Definimos el número de folds
k = 5

# Mezclamos los datos aleatoriamente
indices = np.arange(len(x))
np.random.shuffle(indices)
X = x[indices]
Y = y[indices]


# Divide los datos en k grupos
X_folds = np.array_split(X, k)
Y_folds = np.array_split(Y, k)



# Lista para almacenar los porcentajes de rendimiento
accuracies=[]

# Realizamos k iteraciones
for i in range(k):
    # Selecciona el conjunto de prueba y entrenamiento actual
    X_test = X_folds[i]
    Y_test = Y_folds[i]
    
    X_train = np.concatenate([X_folds[j] for j in range(k) if j != i])
    Y_train = np.concatenate([Y_folds[j] for j in range(k) if j != i])
    
    #Creamos y entrenamos el modelo.

    # Iniciar Regresion
    clasificador = Regresion(700,0.00000001)
    clasificador.train(X_train, Y_train)
    
    # Evalúa el modelo en el conjunto de prueba
    y_pred_test = clasificador.predict(X_test)
    porcentaje_aciertos=acuary(y_pred_test,Y_test)   
    
    # Almacena la precisión en la lista de accuracies
    accuracies.append(porcentaje_aciertos)


# Calcula la precisión promedio de precion en cada K-fold
average_accuracy = np.mean(accuracies)
print(f'Precisión promedio: {average_accuracy:.2f}%')




# Modelo KNN

#### Validacion por division del dataset en 70% entrenamiento 15% validación, 15% prueba.

In [ ]:
# Mezclamos los datos aleatoriamente
indices = np.arange(len(x))
np.random.shuffle(indices)
X = x[indices]
Y = y[indices]

total_samples = len(X)
train = int(0.7 * total_samples)
validation = int(0.15 * total_samples)

# Dividimoss los datos en conjuntos de entrenamiento, validación y prueba
X_train = X[:train]
y_train = Y[:train]

X_val = X[train:train + validation]
y_val = Y[train:train + validation]

X_test = X[train + validation:]
y_test = Y[train + validation:]

#Entrenamos el modelo
clasificador = KNN(k=5)
clasificador.aprendizaje(X_train.T,y_train)

# Evalúa el modelo en el conjunto de validacion
y_val_pred=clasificador.clasificacion(X_val.T)
aciertos_val=acuary(y_val_pred,y_val)

print(f"Precisión en el conjunto de validación: {aciertos_val:.2f}")


# Evalúa el modelo en el conjunto de prueba
y_test_pred=clasificador.clasificacion(X_test.T)
aciertos_prue=acuary(y_test_pred,y_test)

print(f"Precisión en el conjunto de prueba: {aciertos_prue:.2f}")

#### Validacion por K-fold cross validation

In [ ]:
# Definimos el número de folds
k = 5

# Mezclamos los datos aleatoriamente
indices = np.arange(len(x))
np.random.shuffle(indices)
X = x[indices]
Y = y[indices]


# Divide los datos en k grupos
X_folds = np.array_split(X, k)
Y_folds = np.array_split(Y, k)



# Lista para almacenar los porcentajes de rendimiento
accuracies=[]

# Realizamos k iteraciones
for i in range(k):
    # Selecciona el conjunto de prueba y entrenamiento actual
    X_test = X_folds[i]
    Y_test = Y_folds[i]
    
    X_train = np.concatenate([X_folds[j] for j in range(k) if j != i])
    Y_train = np.concatenate([Y_folds[j] for j in range(k) if j != i])
    
    #Creamos y entrenamos el modelo.

    # Iniciar KNN
    clasificador = KNN(k=5)
    clasificador.aprendizaje(X_train.T,Y_train)
    
    # Evalúa el modelo en el conjunto de prueba
    y_pred=clasificador.clasificacion(X_test.T)
    porcentaje_aciertos=acuary(y_pred,Y_test)    
    
    # Almacena la precisión en la lista de accuracies
    accuracies.append(porcentaje_aciertos)


# Calcula la precisión promedio de precion en cada K-fold
average_accuracy = np.mean(accuracies)
print(f'Precisión promedio: {average_accuracy:.2f}%')

# Evaluamos la precision, Recall y  F1-Score
classification_rep = classification_report(Y_test, y_pred)
print("Informe de Clasificación:")
print(classification_rep)

# Modelo SVM
#### Validacion por division del dataset en 70% entrenamiento 15% validación, 15% prueba.

In [ ]:
# Definimos los tamaños de los conjuntos (70% entrenamiento, 15% validación, 15% prueba)
total_samples = len(x)
train = int(0.7 * total_samples)
validation = int(0.15 * total_samples)

# Dividimoss los datos en conjuntos de entrenamiento, validación y prueba
X_train = x[:train]
y_train = y[:train]

X_val = x[train:train + validation]
y_val = y[train:train + validation]

X_test = x[train + validation:]
y_test = y[train + validation:]

# Crear y entrenar el modelo
modelo = SVM(1,300,0.0000001)
modelo.fit(X_train, y_train)

# Validacion
y_pred_val = modelo.prediccion(X_val)
precision_val=acuary(y_val,y_pred_val)

print(f"Precisión en el conjunto de validación: {precision_val:.2f}")

# Predicción
y_pred_test = modelo.prediccion(X_test)
precision_test=acuary(y_test,y_pred_test)

print(f"Precisión en el conjunto de testeo: {precision_test:.2f}")

#### Validacion por K-fold cross validation

In [ ]:
# Definimos el número de folds
k = 5

# Mezclamos los datos aleatoriamente
indices = np.arange(len(x))
np.random.shuffle(indices)
X = x[indices]
Y = y[indices]


# Divide los datos en k grupos
X_folds = np.array_split(X, k)
Y_folds = np.array_split(Y, k)



# Lista para almacenar los porcentajes de rendimiento
accuracies=[]

# Realizamos k iteraciones
for i in range(k):
    # Selecciona el conjunto de prueba y entrenamiento actual
    X_test = X_folds[i]
    Y_test = Y_folds[i]
    
    X_train = np.concatenate([X_folds[j] for j in range(k) if j != i])
    Y_train = np.concatenate([Y_folds[j] for j in range(k) if j != i])
    
    #Creamos y entrenamos el modelo.

    # Iniciar Regresion
    clasificador = SVM(1,300,0.0000001)
    clasificador.fit(X_train, Y_train)
    
    # Evalúa el modelo en el conjunto de prueba
    y_pred_test = clasificador.prediccion(X_test)
    porcentaje_aciertos=acuary(y_pred_test,Y_test)   
    
    # Almacena la precisión en la lista de accuracies
    accuracies.append(porcentaje_aciertos)


# Calcula la precisión promedio de precion en cada K-fold
average_accuracy = np.mean(accuracies)
print(f'Precisión promedio: {average_accuracy:.2f}%') 


# Evaluamos la precision, Recall y  F1-Score
classification_rep = classification_report(Y_test, y_pred)
print("Informe de Clasificación:")
print(classification_rep)

# Modelo Arboles de Desición

In [ ]:
import numpy as np
import pandas as pd
from skimage.io import imread, imshow
from pathlib import Path
import pywt
import pywt.data
from PIL import Image
from sklearn.decomposition import PCA # Para reducir la dimensionalidad
from sklearn.preprocessing import StandardScaler # Para hacer la normalización

def redimensionar(path):
   imagenes=[]
   entries = Path(path)
   for entry in entries.iterdir():
      imagenes.append(path + entry.name)

   # Tamaño deseado para todas las imágenes
   ancho_deseado = 300
   alto_deseado = 200

   # Carpeta de destino para las imágenes redimensionadas
   carpeta_destino = "imagenes_1"

   # Crea la carpeta de destino si no existe
   if not os.path.exists(carpeta_destino):
      os.makedirs(carpeta_destino)

   # Recorre la lista de imágenes y redimensiona y guarda cada una
   for ruta_imagen in imagenes:
      imagen = Image.open(ruta_imagen)
      
      # Redimensiona la imagen al tamaño deseado con interpolación ANTIALIAS
      imagen_redimensionada = imagen.resize((ancho_deseado, alto_deseado), Image.ADAPTIVE)
      
      # Obtiene el nombre de archivo sin la ruta
      nombre_archivo = os.path.basename(ruta_imagen)
      
      # Define la ruta de archivo de destino en la carpeta de destino
      ruta_destino = os.path.join(carpeta_destino, nombre_archivo)
      
      # Guarda la imagen redimensionada en la carpeta de destino
      imagen_redimensionada.save(ruta_destino)   


# Partición de la imagen en su esquina superior izquierda
def Get_Feacture(picture, cortes):
    LL = picture.copy()
    for i in range(cortes): # Se corta la cantidad de veces determinada (2)
        LL, (LH, HL, HH) = pywt.dwt2(LL, 'haar')
    return LL.flatten().tolist() # Se convierte esta matriz representativa en un vector característico

def Cargar_dataset_arbol():

    path = './imagenes_1/'

    vectores_caracteristicos = []

    entries = Path(path)

    # Iteramos la lista de directorios
    for entry in entries.iterdir():
        imagen = path + entry.name
        picture = imread(imagen)
        vector_caracteristico = Get_Feacture(picture, 2) # Se extrae el vector característico
        vector_caracteristico.append(entry.name[0:3]) # Se agrega el label
        vectores_caracteristicos.append(vector_caracteristico) # Agregamos ese nuevo vector característico

    # Convertimos a numpy los datos de entrada
    X = np.array(vectores_caracteristicos)[:, :-1]  # Elimina la última columna (labels) para dejar solo los datos de entrada

    # Aplicamos PCA para reducir la dimensionalidad
    pca = PCA(n_components=200) # Elegimos el número de componentes principales
    X_pca = pca.fit_transform(X)

    # Normalizamos los datos
    scaler = StandardScaler()
    X_pca_normalized = scaler.fit_transform(X_pca)


    # Obtenemos la lista de etiquetas
    labels = np.array(vectores_caracteristicos)[:, -1]

    return X_pca_normalized,labels


# Redimensionamos las imagenes para trabajar todas con el mismo tamaño
redimensionar("./images/")

# Definimos el dataset
x,y= cargar_dataset()

x=np.array(x)
y=np.array(y)
